# Практика 8: Методы защиты от атак на модели ИИ
# Выполнил студент группы ББМО-01-23 Морин А.А.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

import tensorflow as tf
from tensorflow.keras.layers import Activation
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical


Загрузка данных MNIST

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

train_images_1 = train_images[:1000]
train_labels_1 = train_labels[:1000]


Защита с помощью Adversarial Training

Функция для FGSM атаки

In [3]:
def fgsm_attack(image, epsilon, gradient):
    perturbation = epsilon * np.sign(gradient)
    adversarial_image = image + perturbation
    adversarial_image = np.clip(adversarial_image, 0, 1)
    return adversarial_image

Функция для генерации противоречивых примеров

In [4]:
def generate_adversarial_examples(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        image = tf.convert_to_tensor(images[i].reshape((1, 28, 28, 1)), dtype=tf.float32)
        label = tf.convert_to_tensor(labels[i].reshape((1, 10)), dtype=tf.float32)

        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.categorical_crossentropy(label, prediction)
        gradient = tape.gradient(loss, image)
        adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
        adversarial_images.append(adversarial_image.reshape(28, 28))

    return np.array(adversarial_images)


Создание модели

In [5]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

Обучение модели с противоречивыми примерами

In [6]:
def adversarial_training(model, train_image_1, train_labels, epsilon):
    for epoch in range(10):
        for i in range(0, len(train_images_1), 32):
            batch_images = train_images_1[i:i+32]
            batch_labels = train_labels_1[i:i+32]
            adversarial_images = generate_adversarial_examples(model, batch_images, batch_labels, epsilon)
            combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
            combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)
            model.train_on_batch(combined_images, combined_labels)

Инициализация модели

In [7]:
model = create_model()

d:\anaconda\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Тренировка модели с защитой на противоречивых примерах

In [8]:
adversarial_training(model, train_images_1, train_labels_1, epsilon=0.1)

Сохранение защищенной модели

In [9]:
model.save('adversarial_trained_model.h5')

Градиентная маскировка (Gradient Masking)

Обновление модели для градиентной маскировки

In [11]:
def create_masked_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10),
        Activation('softplus')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

Обучение модели с градиентной маскировкой

In [12]:
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8686 - loss: 0.4788
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9601 - loss: 0.1326
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9726 - loss: 0.0899
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9812 - loss: 0.0627
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9846 - loss: 0.0519


Сохранение модели с градиентной маскировкой

In [13]:
masked_model.save('masked_model.h5')

Регуляризация и нормализация для повышения устойчивости

Модель с регуляризацией и нормализацией

In [14]:
def create_regularized_model():
 model = tf.keras.Sequential([
 tf.keras.layers.Flatten(input_shape=(28, 28)),
 tf.keras.layers.Dense(128, activation='relu',
kernel_regularizer=tf.keras.regularizers.l2(0.01)),
 tf.keras.layers.Dropout(0.5),tf.keras.layers.BatchNormalization(),
 tf.keras.layers.Dense(10, activation='softmax')
 ])
 model.compile(optimizer='adam', loss='categorical_crossentropy',
metrics=['accuracy'])
 return model

Обучение модели с регуляризацией и нормализацией

In [15]:
regularized_model = create_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.7899 - loss: 1.3405
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8676 - loss: 0.6290
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8638 - loss: 0.6253
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8639 - loss: 0.6237
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8588 - loss: 0.6276


Сохранение модели с регуляризацией

In [16]:
regularized_model.save('regularized_model.h5')

Оценка моделей на противоречивых примерах

Adversarial Training

In [17]:
# Загрузка модели
protected_model = tf.keras.models.load_model('adversarial_trained_model.h5')
# Генерация противоречивых примеров для тестовых данных
adversarial_test_images = generate_adversarial_examples(protected_model,
test_images, test_labels, epsilon=0.1)
# Оценка защищенной модели на противоречивых примерах
test_loss, test_acc = protected_model.evaluate(adversarial_test_images,test_labels)
print(f'Accuracy of protected model on adversarial examples: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step - accuracy: 0.5527 - loss: 1.3350
Accuracy of protected model on adversarial examples: 0.6065000295639038


In [18]:
# Загрузка модели
regularized_model = tf.keras.models.load_model('regularized_model.h5')
masked_model = tf.keras.models.load_model('masked_model.h5')

# Оценка модели с регуляризацией и нормализацией на противоречивых примерах
test_loss, test_acc = regularized_model.evaluate(adversarial_test_images,test_labels)
print(f'Accuracy of regularized model on adversarial examples: {test_acc}')

# Оценка модели с градиентной маскировкой на противоречивых примерах
test_loss, test_acc = masked_model.evaluate(adversarial_test_images,test_labels)
print(f'Accuracy of masked model on adversarial examples: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - accuracy: 0.6584 - loss: 1.1171
Accuracy of regularized model on adversarial examples: 0.7056000232696533
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 792us/step - accuracy: 0.6503 - loss: 1.1788
Accuracy of masked model on adversarial examples: 0.6970000267028809


# Выводы

Тестирование на противоречивых примерах показало, что различные методы защиты улучшают устойчивость модели к атакам.

Adversarial Training продемонстрировал точность 60.6% на противоречивых примерах, что выше по сравнению с исходной моделью.

Модель с регуляризацией достигла точности 70.5%, что указывает на эффективность регуляризации как метода защиты.

Маскирующая модель показала точность 69.7% на противоречивых примерах.